# Predicción lineal

 En la figura se observa un filtro forward predictor con una estructura FIR

<img src="predictor.png" alt="Predictor lineal"/>

Las entradas a los taps son $u(i-1), u(i-2), ..., u(i-M)$. La idea de este filtro es predecir el valor siguiente de la señal $u(i)$, denotado $\hat{u}(i)$.

El objetivo es encontrar el vector de taps *${\hat{w}}$* que minimice la suma cuadrática de los errores de predicción, $e_f$

$$e_f = \sum_{i=M+1}^{N} \left|f_M(i)\right|^2$$

#### Encontrar:

1. La matriz de $M$ por $M$ de correlación los taps de entrada 

2. El vector de correlación cruzada $M$ por $1$ entre los taps de entrada del predictor y la respuesta deseada $u(i)$ 

3. El valor mínimo de $e_f$

4. El vector de taps *${\hat{w}}$*

5. Una vez obtenida la ecuación del filtro predictor, calcular el filtro para $N$ valores de una realización de la siguiente señal:

$$u(i) = sen(2\pi\omega_0i + \phi_0) + 0.02n(i)$$

siendo $\omega_0 = 3, \phi_0 = \pi / 4$ y $n(i)$ un ruido gaussiano de varianza unitaria. Tomar N = 100 y M = 4

6. Ver cómo se comporta el filtro (es decir, su capacidad de predicción) para otras realizaciones del mismo proceso.

#### Consejos orientativos:
* Pensar a $N$ como un valor mayor a $M$. Es decir, $N$ representa la cantidad TOTAL de puntos de $u(i)$ y $M$ es la cantidad de taps del filtro. En un momento dado, hay $M$ puntos de $u(i)$ dentro del filtro.
* Podemos pensar que el primer valor válido de $u$ es $u(1)$, es decir la secuencia empieza $u(1), u(2), \dots ,u(N)$
* El filtro produce resultados desde $i = M+1$ hasta $i = N$, es decir, $N-M$ predicciones.
* Al ser las señales reales, los valores hermíticos de los vectores o matrices son los traspuestos.
* Para multiplicación de matrices, utilizar @. Por ejemplo:

In [9]:
import numpy as np
a = np.array([[1,2,3],[2,3,1]])
b = np.array([-1,4,3])
c = a @ b
c

array([16, 13])

## Respuestas

# R1. 
La matriz de MxM de autocorrelacion de los taps de entrada es; 

<img src="matriz_correlacion2.png" alt="matriz correlacion" width=500/>

donde $\phi(t,k)$ son los coeficientes de autocorrelacion segun:

<img src="fi.png" alt="fi" width=200/>

pero por convenientica matematica se define una matriz A hermitica, tal que:

<img src="a_hermitica.png" alt="a_hermitica" width=400/>

In [13]:
# genera la matriz A hermitica basada en el vector de entrada Nx1 y el numero de coeficientes M
# elegido. resultando en una matriz de M filas y N (N-M+1) columnas              
def generateAHermitica(self,u,M):                                                
    N=u.size-M+1                                                                 
    aH=np.zeros((M,N))                                                              
    for i in range(M):                                                           
        aH[i]=u[M-i-1:M-i-1+N]                                                   
    return aH 

entonces $\phi$ sera:

<img src="fi_a_hermitica.png" alt="fi_a_hermitica" width=150/>

In [15]:
#simplemente calcula la matriz hermitica de otra matriz, para hacer mas legible el codigo y
#reutilizar la funcion repetidamente, si entra MxN sale NxM                      
def calcHermitica(self,aH):                                                      
    return aH.transpose().conjugate()                                            
                                                                                 
#calcula la matriz phi de autocorrelacion, basado en la matriz hermitica, es un calculo simple
# pero de nuevo se lo define como funcion por modularidad, el resultado sera una matriz de MxM
def phi(self,aH):                                                                
    return aH@self.calcHermitica(aH) 

# R2.
Luego el vector Z de correlacion cruzada entre los taps de entrada del predictor y la respuesta deseada sera:

<img src="z.png" alt="z" width=350/>
<img src="z2.png" alt="z2" width=150/>

In [16]:
#calcula el vector correlacion cruzada z, que es simplemente A hermitica por el vector d. el
#resultado sera de dimension Mx1                                                 
def z(self,aH,d):                                                                
    return aH@d 

# R3.
El valor minimo e, como el minimo error cuadratico medio entre la senial obtenida y la deseada sera:


<img src="e.png" alt="e" width=350/>

In [18]:
#calcula e (1x1) que es el error cuadratico medio entre el vector deseado y lo obtenido a
#la salida del filtro                                                            
def e(self,aH,d):                                                                
    return self.calcHermitica(d)@d - self.w(aH,d)@self.z(aH,d)                   
                                                                 

# R4.
El vector de taps del filtro se calcula mediante:


<img src="w.png" alt="w" width=350/>

In [19]:
#calcula el vector w que seran los parametros del filtro Mx1                     
def w(self,aH,d):                                                                
    return np.linalg.inv(self.phi(aH))@self.z(aH,d)

# R5.
experimentacion: